# NOSQL - Trabalho prátrico

Adicionar descrição/introdução


## Informação relativa à base de dados

- database:trabalho_pratico
- username: nosql
- password: nosql2024


Identificaram-se 4 coleções: ....

A etapa inicial do trabalho consistiu em estabelecer uma conexão entre o Python e a base de dados Oracle. Isso implicou a especificação do utilizador, palavra-passe, anfitrião, porta e SID do Oracle, previamente definidos. Assim, tornou-se possível executar consultas e recuperar dados da base de dados, possibilitando a visualização de todas as informações lá contidas.

# Librarias e packages necessários

In [1]:
#pip install pymongo
#pip install cx_Oracle
#pip install oracledb
#pip install mysql-connector-python

SyntaxError: invalid syntax (2409844679.py, line 1)

# Introdução

A base deste trabalho é um conjunto de 17 tabelas que retratam um Hospital. 

O objetivo principal deste trabalho passa por explorar diferentes paradigmas de base de dados, tendo como referências dois tipos de modelos, um modelo relacional e um modelo não relacional. 

Em particular, o modelo não relacional será o mais explorado, sendo que, com auxílio de código (maioritariamente em python) iremos, através de uma estratégia definida e explicada posteriormente, "converter" o modelo relacional já dado em modelos não relacionais explorando as bases de dados MongoDB e Neo4j e seus respetivos paradigmas. 

In [1]:
import oracledb
import getpass
import cx_Oracle

# Pedir senha
#password = getpass.getpass(prompt="Enter Oracle password: ")

# Conectar ao Oracle
#connection = oracledb.connect(user="nosql", password=password, dsn="localhost/xe")
#cursor = connection.cursor()

In [2]:
#MACM1 Rodrigo

connection = oracledb.connect(user="EU", password="12345678", dsn="localhost/ORCLCDB")
cursor = connection.cursor()

# MONGO

### Pacientes

In [35]:
import pymongo
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

db = client.Hospital

#Patients = db.create_collection("Patients")
#Services = db.create_collection("Services")
#Appointments = db.create_collection("Appointments")
#Staff = db.create_collection("Staff")


collection_p = db.Patients
collection_a = db.Appointments
collection_s = db.Services
collection_sa = db.Staff



Relações Entre as Coleções
As quatro coleções estão interligadas por meio de referências a IDs de documentos em outras coleções, semelhante às chaves estrangeiras em bancos de dados relacionais. As principais relações são:

Pacientes e Funcionários: Os IDs de funcionários (como médicos, enfermeiros e técnicos) são referenciados em documentos de pacientes e hospitalizações para indicar o responsável pelo atendimento.

Pacientes e Agendamentos: Cada agendamento contém uma referência ao ID do paciente e ao ID do médico que realizará a consulta. Isso permite rastrear quais consultas estão associadas a cada paciente e médico.

Pacientes e Atendimentos: As hospitalizações referenciam o ID do paciente, conectando diretamente os registros de hospitalização ao paciente correspondente. Além disso, incluem referências a exames laboratoriais e aos técnicos responsáveis pelos testes.

Funcionários e Atendimentos: As hospitalizações e os exames laboratoriais incluem referências aos IDs de enfermeiros e técnicos, conectando os funcionários aos procedimentos realizados durante a hospitalização.

Exemplo de Interação Entre Coleções
Imagine um paciente chamado John Doe (ID: P001) que tem um agendamento (ID: A001) com o Dr. Smith (ID: S001) no dia 10 de maio de 2023. Após a consulta, John é hospitalizado (ID: E001) e recebe tratamento do enfermeiro Jane (ID: N001). Durante sua hospitalização, ele passa por um exame de sangue realizado pelo técnico Tom (ID: T001).

O documento de paciente para John Doe na coleção Patients incluirá seus dados pessoais e históricos médicos.
O documento de agendamento na coleção Appointments referenciará John Doe (patientId: P001) e Dr. Smith (doctorId: S001).
O documento de hospitalização na coleção Hospitalizations incluirá referências ao paciente John Doe (patientId: P001), ao enfermeiro Jane (responsibleNurseId: N001) e ao técnico Tom (labScreenings.technicianId: T001).
Conclusão
Essa estrutura de coleções no MongoDB permite um armazenamento organizado e relacional dos dados hospitalares, facilitando o acesso e a manipulação das informações através das referências entre os documentos. Essa abordagem melhora a integridade dos dados e a eficiência das operações de busca e atualização no sistema hospitalar.

In [1]:
#### Parte do Duarte

In [15]:
def extract_data_from_oracle():

    # Pedir senha
    password = getpass.getpass(prompt="Enter Oracle password: ")

    # Conectar ao Oracle
    connection = oracledb.connect(user="EU", password=password, dsn="localhost/ORCLCDB")
    cursor = connection.cursor()

# Consulta SQL para extrair os dados de HOSPITAL_PATIENT
    sql_patient = """
    SELECT
        idpatient,     
        patient_fname, 
        patient_lname, 
        blood_type, 
        phone,         
        email,         
        gender,        
        policy_number, 
        BIRTHDAY
    FROM
        patient
"""
    cursor.execute(sql_patient)
    patients = cursor.fetchall()

# Consulta SQL para extrair os dados de HOSPITAL_INSURANCE
    sql_insurance = """
        SELECT 
            policy_number,  
            provider,       
            insurance_plan, 
            co_pay,         
            coverage,       
            maternity,      
            dental,         
            optical     
        FROM 
            insurance
    """
    
    cursor.execute(sql_insurance)
    insurances = cursor.fetchall()
    
 # Consulta SQL para extrair os dados de HOSPITAL_MEDICAL_HISTORY
    sql_MEDICAL_HISTORY = """
        SELECT 
            record_id,   
            record_date, 
            idpatient   
        FROM 
            MEDICAL_HISTORY
    """
    cursor.execute(sql_MEDICAL_HISTORY)
    MEDICAL_HISTORYs = cursor.fetchall()

    # Consulta SQL para extrair os dados de HOSPITAL_EMERGENCY_CONTACT
    sql_emergency_contact = """
        SELECT 
            contact_name, 
            phone,        
            relation,    
            idpatient  
        FROM 
            emergency_contact
    """
    cursor.execute(sql_emergency_contact)
    lab_emergency_contact = cursor.fetchall()

    # Fechar conexão
    cursor.close()
    connection.close()

    # Inserir os dados na coleção MongoDB
    insert_data_to_mongodb(patients, insurances, MEDICAL_HISTORYs, lab_emergency_contact)

In [16]:
# Função para inserir os dados no MongoDB
def insert_data_to_mongodb(patients, insurances, MEDICAL_HISTORYs, lab_emergency_contact):
    
    # Conectar ao MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['Hospital']  # Substitua 'nome_do_banco_de_dados' pelo nome do seu banco de dados no MongoDB
    collection = db['Patients']

    # Formatar os dados e inserir na coleção MongoDB
    for patient in patients:
        patient_data = {
            "idepisode": patient[0],
            "patient_fname": patient[1],
            "patient_lname" : patient[2] ,   
            "blood_type": patient[3],
            "phone"   : patient[4],    
            "email"  : patient[5],     
            "gender"   : patient[6],   
            "policy_number": patient[7],
            "BIRTHDAY": patient[8],
        }
        collection.insert_one(patient_data)

    for insurance in insurances:
        insurance_data = {
            "policy_number" : insurance[0],
            "provider"  : insurance[1],     
            "insurance_plan" : insurance[2],
            "co_pay"      : insurance[3],   
            "coverage"   : insurance[4],    
            "maternity"  : insurance[5],    
            "dental"   : insurance[6],      
            "optical" : insurance[7],
        }
        collection.insert_one(insurance_data)

    for MEDICAL_HISTORY in MEDICAL_HISTORYs:
        MEDICAL_HISTORY_data = {
            "record_id"  : MEDICAL_HISTORY[0], 
            "record_date" : MEDICAL_HISTORY[1],
            "idpatient" : MEDICAL_HISTORY[2],
        }
        collection.insert_one(MEDICAL_HISTORY_data)

    for emergency_contact in lab_emergency_contact:
        emergency_contact_data = {
            "contact_name ": emergency_contact[0],
            "phone": emergency_contact[1],        
            "relation": emergency_contact[2],    
            "idpatient": emergency_contact[3],
        }
        collection.insert_one(emergency_contact_data)

    # Fechar conexão
    client.close()

# Chamar a função principal para extrair dados do Oracle e inseri-los no MongoDB
extract_data_from_oracle()

### Agendamento


In [17]:
import getpass
import oracledb

def extract_data_from_oracle():
    # Pedir senha
    password = getpass.getpass(prompt="Enter Oracle password: ")

    # Conectar ao Oracle
    connection = oracledb.connect(user="EU", password=password, dsn="localhost/ORCLCDB")
    cursor = connection.cursor()

    # Consulta SQL para extrair os dados de HOSPITAL_APPOINTMENT
    sql_appointment = """
        SELECT 
            scheduled_on,
            appointment_date,
            appointment_time,
            iddoctor,
            idepisode
        FROM 
            APPOINTMENT
    """
    cursor.execute(sql_appointment)
    appointments = cursor.fetchall()

    # Fechar conexão
    cursor.close()
    connection.close()

    # Inserir os dados na coleção MongoDB
    insert_data_to_mongodb(appointments)

In [19]:
# Função para inserir os dados na coleção 'HOSPITAL_APPOINTMENT'
def insert_data_to_mongodb(appointments):
    
    # Conectar ao MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['Hospital']  # Substitua 'nome_do_banco_de_dados' pelo nome do seu banco de dados no MongoDB
    collection = db['Appointments']

    # Formatar os dados e inserir na coleção MongoDB
    for appointment in appointments:
        appointment_data = {
            "scheduled_on": appointment[0],
            "appointment_date": appointment[1],
            "appointment_time": appointment[2],
            "iddoctor": appointment[3],
            "idepisode": appointment[4],
        }
        collection.insert_one(appointment_data)

    # Fechar conexão
    client.close()

# Chamar a função principal para extrair dados do Oracle e inseri-los no MongoDB
extract_data_from_oracle()


### Atendimento

In [23]:
def extract_data_from_oracle():

    # Pedir senha
    password = getpass.getpass(prompt="Enter Oracle password: ")

    # Conectar ao Oracle
    connection = oracledb.connect(user="EU", password=password, dsn="localhost/ORCLCDB")
    cursor = connection.cursor()

# Consulta SQL para extrair os dados de HOSPITAL_EPISODE
    sql_episode = """
        SELECT 
            idepisode,
            patient_idpatient
        FROM 
            episode
    """
    cursor.execute(sql_episode)
    episodes = cursor.fetchall()

# Consulta SQL para extrair os dados de HOSPITAL_ROOM
    sql_room = """
        SELECT 
            idroom,
            room_type,
            room_cost
        FROM 
            room
    """
    cursor.execute(sql_room)
    rooms = cursor.fetchall()
 # Consulta SQL para extrair os dados de HOSPITAL_BILL
    sql_bill = """
        SELECT 
            idbill,
            room_cost,
            test_cost,
            other_charges,
            total,
            idepisode,
            registered_at,
            payment_status
        FROM 
            bill
    """
    cursor.execute(sql_bill)
    bills = cursor.fetchall()

    # Consulta SQL para extrair os dados de HOSPITAL_LAB_SCREENING
    sql_lab_screening = """
        SELECT 
            lab_id,
            test_cost,
            test_date,
            idtechnician,
            episode_idepisode
        FROM 
            lab_screening
    """
    cursor.execute(sql_lab_screening)
    lab_screenings = cursor.fetchall()

    # Fechar conexão
    cursor.close()
    connection.close()

    # Inserir os dados na coleção MongoDB
    insert_data_to_mongodb(episodes, rooms, bills, lab_screenings)

In [24]:
# Função para inserir os dados no MongoDB
def insert_data_to_mongodb(episodes, rooms, bills, lab_screenings):
    
    # Conectar ao MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['Hospital']  # Substitua 'nome_do_banco_de_dados' pelo nome do seu banco de dados no MongoDB
    collection = db['Services']

    # Formatar os dados e inserir na coleção MongoDB
    for episode in episodes:
        episode_data = {
            "idepisode": episode[0],
            "patient_idpatient": episode[1],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(episode_data)

    for room in rooms:
        room_data = {
            "idroom": room[0],
            "room_type": room[1],
            "room_cost": room[2],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(room_data)

    for bill in bills:
        bill_data = {
            "idbill": bill[0],
            "room_cost": bill[1],
            "test_cost": bill[2],
            "other_charges": bill[3],
            "total": bill[4],
            "idepisode": bill[5],
            "registered_at": bill[6],
            "payment_status": bill[7],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(bill_data)

    for lab_screening in lab_screenings:
        lab_screening_data = {
            "lab_id": lab_screening[0],
            "test_cost": lab_screening[1],
            "test_date": lab_screening[2],
            "idtechnician": lab_screening[3],
            "episode_idepisode": lab_screening[4],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(lab_screening_data)

    # Fechar conexão
    client.close()

# Chamar a função principal para extrair dados do Oracle e inseri-los no MongoDB
extract_data_from_oracle()

### STAFF

In [26]:
def extract_data_from_oracle():
    # Pedir senha
    password = getpass.getpass(prompt="Enter Oracle password: ")

    # Conectar ao Oracle
    connection = oracledb.connect(user="EU", password=password, dsn="localhost/ORCLCDB")
    cursor = connection.cursor()

    # Consulta SQL para extrair os dados de STAFF
    sql_staff = """
        SELECT 
            EMP_ID,
            EMP_FNAME,
            EMP_LNAME,
            DATE_JOINING,
            DATE_SEPERATION,
            EMAIL,
            ADDRESS,
            SSN,
            IDDEPARTMENT,
            IS_ACTIVE_STATUS
        FROM 
            STAFF
    """
    cursor.execute(sql_staff)
    staff = cursor.fetchall()

    # Fechar conexão
    cursor.close()
    connection.close()

    # Inserir os dados na coleção MongoDB
    insert_data_to_mongodb(staff)   # Função para inserir os dados na coleção 'Staff'

def insert_data_to_mongodb(staffs):
    # Conectar ao MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['Hospital']  # Substitua 'Hospital' pelo nome do seu banco de dados no MongoDB
    collection = db['Staff']

    # Formatar os dados e inserir na coleção MongoDB
    for staff in staffs:
        staff_data = {
            "_id": staff[0],  # Use o ID do funcionário como o ID do documento
            "firstName": staff[1],
            "lastName": staff[2],
            "joiningDate": staff[3],
            "separationDate": staff[4],
            "email": staff[5],
            "address": staff[6],
            "ssn": staff[7],
            "departmentId": staff[8],
            "isActiveStatus": staff[9]
        }
        collection.insert_one(staff_data)

    # Fechar conexão
    client.close()

# Chamar a função principal para extrair dados do Oracle e inseri-los no MongoDB
extract_data_from_oracle()

## INDEX

In [29]:

def create_index_on_staff_collection():
    # Conectar ao MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['Hospital']
    collection_p = db['Patients']
    collection_a = db['Appoitments']
    collection_s = db['Services']
    collection_sa = db['Staff']

    # Criar índice na coleção 'Staff' no campo '_id'
    collection_p.create_index([("idepisode", 1)])
    collection_a.create_index([("scheduled_on", 1)])
    collection_s.create_index([("idbill", 1)])
    collection_sa.create_index([("_id", 1)])

    # Fechar conexão
    client.close()


In [ ]:
#se por ventura for necessário apagar um idex

#Hospital.drop_index("")

In [36]:
indexes= collection_p.list_indexes()
for index in indexes:
    print(index)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


## Procedure